In [1]:
from pathlib import Path
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.utils.nyc_taxi_zones import TaxiZones, plot_zones
from src.utils.nyc_trip_loader import NYCTripData

BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / 'data'
YELLOW_DIR = DATA_DIR / 'yellow_taxi_trip_records'
GREEN_DIR = DATA_DIR / 'green_taxi_trip_records'
FHV_DIR = DATA_DIR / 'for_hire_vehicle_trip_records'
OUTPUT_DIR = BASE_DIR / 'output'

Loading precomputed graph from /mnt/ufs18/home-207/indibimu/repos/demand-forecast/demand-forecast/data/taxi_zones/precomputed_graphs/G_nyc_3ec450e2f75c938546192cae174c0cb4.graphml
Graph already exists at /mnt/ufs18/home-207/indibimu/repos/demand-forecast/demand-forecast/data/taxi_zones/precomputed_graphs/G_nyc_3ec450e2f75c938546192cae174c0cb4.graphml. Did not overwrite.


In [2]:
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data, InMemoryDataset #DataLoader
from torch_geometric.loader import DataLoader

In [3]:
val_data = NYCTripData('2021-09','2022-08', dataset='fhvhv')
train_data = NYCTripData('2022-09','2023-08', dataset='fhvhv')
test_data = NYCTripData('2023-09','2024-08', dataset='fhvhv')

/mnt/ufs18/home-207/indibimu/repos/demand-forecast/demand-forecast/src/utils/nyc_trip_loader.py:185: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_data.fillna(method='ffill', inplace=True)
/mnt/ufs18/home-207/indibimu/repos/demand-forecast/demand-forecast/src/utils/nyc_trip_loader.py:185: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_data.fillna(method='ffill', inplace=True)
/mnt/ufs18/home-207/indibimu/repos/demand-forecast/demand-forecast/src/utils/nyc_trip_loader.py:185: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_data.fillna(method='ffill', inplace=True)


In [13]:
temporal_base_idx = ~train_data.feature_names.isin( ['PULocationID', 'x', 'y', 'PU_count'])
GData = from_networkx(TaxiZones.G_nyc , group_node_attrs=['pos_x', 'pos_y'])
Graph_base = Data(edge_index=GData.edge_index, pos=GData.x) 
temporal_base = train_data.trip_data[0, :, temporal_base_idx]
frcst_w = 1
obs_w = 24
dtype = torch.float32
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
data_list = []
for i in range(obs_w+1, temporal_base.shape[1]-frcst_w-obs_w-1, 1):
    obs_window = torch.tensor(train_data.trip_data[:,i:i+obs_w-1,29].astype(float))
    obs_features = train_data.trip_data[:,i+obs_w,:].astype(float)
    obs_window = torch.cat([torch.tensor(obs_window), torch.tensor(obs_features)], dim=1)
    forecast_window = torch.tensor(train_data.trip_data[:,i+obs_w:i+obs_w+frcst_w,29].astype(float))

    data_list.append(Data(edge_index=Graph_base.edge_index.to(device=device),
                        pos=torch.tensor(Graph_base.pos).to(dtype=dtype, device=device),
                        x=obs_window.to(dtype=dtype, device=device),
                        y=forecast_window.to(dtype=dtype, device=device)))

train_loader = DataLoader(data_list, batch_size=32, shuffle=True)

/tmp/ipykernel_3523062/3967254918.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs_window = torch.cat([torch.tensor(obs_window), torch.tensor(obs_features)], dim=1)
/tmp/ipykernel_3523062/3967254918.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos=torch.tensor(Graph_base.pos).to(dtype=dtype, device=device),


In [6]:
from torch.nn import Linear, ReLU
from torch_geometric.nn import Sequential, GCNConv
import torch.nn.functional as F

In [18]:
num_node_features = 53
hidden_channels = 100

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, frcst_w)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.conv2(x, edge_index)
        return x


model = GCN(hidden_channels=hidden_channels)
print(model)

GCN(
  (conv1): GCNConv(53, 100)
  (conv2): GCNConv(100, 1)
)


In [19]:
model = GCN(hidden_channels=16).to(device=device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.MSELoss(reduction='mean')

def train():
    model.train()
    batch_loss = 0
    for data in train_loader:  # Iterate over each mini-batch.
        out = model(data.x, data.edge_index)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss solely based on the training nodes.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        batch_loss += loss.item()
    return batch_loss

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    forecast = out.argmax(dim=1)  # Use the class with highest probability.
    loss = criterion(out, data.y).item()
    return loss

for epoch in range(1, 51):
    loss = train()
    # test_loss = test()
    print(f'Epoch: {epoch:03d}, Training loss: {loss:.4f}')

Epoch: 001, Training loss: 7041956.9258
Epoch: 002, Training loss: 5022552.8770
Epoch: 003, Training loss: 4339505.3721
Epoch: 004, Training loss: 3793263.6621
Epoch: 005, Training loss: 3385391.2578
Epoch: 006, Training loss: 3085408.0850
Epoch: 007, Training loss: 2858202.7407
Epoch: 008, Training loss: 2682825.9673
Epoch: 009, Training loss: 2554908.7705
Epoch: 010, Training loss: 2456330.3525
Epoch: 011, Training loss: 2378521.3354
Epoch: 012, Training loss: 2315816.9409
Epoch: 013, Training loss: 2269495.5684
Epoch: 014, Training loss: 2242202.2993
Epoch: 015, Training loss: 2222450.1689
Epoch: 016, Training loss: 2196036.8838
Epoch: 017, Training loss: 2182746.0010
Epoch: 018, Training loss: 2177295.9253
Epoch: 019, Training loss: 2164377.7188
Epoch: 020, Training loss: 2166617.4961
Epoch: 021, Training loss: 2156806.5930
Epoch: 022, Training loss: 2158814.9785
Epoch: 023, Training loss: 2162595.2246
Epoch: 024, Training loss: 2155039.9810
Epoch: 025, Training loss: 2150380.8584


In [10]:
help(torch.nn.MSELoss)

Help on class MSELoss in module torch.nn.modules.loss:

class MSELoss(_Loss)
 |  MSELoss(size_average=None, reduce=None, reduction: str = 'mean') -> None
 |
 |  Creates a criterion that measures the mean squared error (squared L2 norm) between
 |  each element in the input :math:`x` and target :math:`y`.
 |
 |  The unreduced (i.e. with :attr:`reduction` set to ``'none'``) loss can be described as:
 |
 |  .. math::
 |      \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
 |      l_n = \left( x_n - y_n \right)^2,
 |
 |  where :math:`N` is the batch size. If :attr:`reduction` is not ``'none'``
 |  (default ``'mean'``), then:
 |
 |  .. math::
 |      \ell(x, y) =
 |      \begin{cases}
 |          \operatorname{mean}(L), &  \text{if reduction} = \text{`mean';}\\
 |          \operatorname{sum}(L),  &  \text{if reduction} = \text{`sum'.}
 |      \end{cases}
 |
 |  :math:`x` and :math:`y` are tensors of arbitrary shapes with a total
 |  of :math:`n` elements each.
 |
 |  The mean operation still

In [ ]:
class NYCGraphDataset(InMemoryDataset):
    def __init__(self, root, nyc_trip_data,
                 transform=None,
                 pre_transform=None,
                 pre_filter=None,
                 forecast_window=4,
                 observed_window=4):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.nyc = nyc_trip_data
        temporal_base_idx = ~nyc_trip_data.feature_names.isin( ['PULocationID', 'x', 'y', 'PU_count'])
        GData = from_networkx(TaxiZones.G_nyc , group_node_attrs=['pos_x', 'pos_y'])
        self.Graph_base = Data(edge_index=GData.edge_index, pos=GData.x) 
        self.temporal_base = nyc_trip_data.trip_data[0, :, temporal_base_idx]
        self.fcst_w = forecast_window
        self.obs_w = observed_window
        dta = []
        for i in range(self.temporal_base.shape[1]-self.obs_w):
            obs_window = self.nyc.trip_data[:,i:i+self.obs_w,:]
            forecast_window = self.nyc.trip_data[:,i+self.obs_w:i+self.fcst_w,29]
            dta.append(Data(edge_index=self.Graph_base.edge_index, pos=self.Graph_base.x, x=obs_window, y=forecast_window))
        self.data = dta
        # self.collate(dta)

    def get(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)
    

# train_g_data = NYCGraphDataset(DATA_DIR, train_data, forecast_window=4, observed_window=4)
# print(train_g_data)

